---

###### *Document créé par Jean Lamerenx dans le cadre des JIREC 2024*
---


Ce script génère une animation qui permet de visualiser les points de tangence au sein d'une structure cristallographique de type **cubique à faces centrées**.

La nature des sites interstitiels occupés peut être contrôlée, ainsi que le **rayon des sphères** qui les occupent.

Enfin, divers **plans de coupe** sont proposés.



In [2]:
import matplotlib.pyplot as plt
import numpy as np
from itertools import combinations, product
import ipywidgets as ipw


#Coordonnées des sphères dans le réseau CFC (hôte + sites interstitiels)
coord_hote = [[0,0,0],[0,0,1],[0,1,0],[0,1,1],[1,0,0],[1,0,1],[1,1,0],[1,1,1],
             [.5,.5,0],[.5,.5,1],[.5,0,.5],[.5,1,.5],[0,.5,.5],[1,.5,.5]]

def sites(type_site) :
  if type_site == 'aucun' :
    coord_sites = []
  elif type_site == 'octaédriques' :
    coord_sites = [[0,0,.5],[.5,0,0],[.5,0,1],[1,0,.5],
                   [0,1,.5],[.5,1,0],[.5,1,1],[1,1,.5],
                   [0,.5,0],[0,.5,1],[1,0.5,0],[1,.5,1],
                   [.5,.5,.5]]
  elif type_site == 'tétraédriques' :
    coord_sites = [[.25,.25,.25],[.75,.25,.25],[.25,.75,.25],[.25,.25,.75],
                   [.75,.75,.25],[.75,.25,.75],[.25,.75,.75],[.75,.75,.75]]
  return coord_sites


#Préparation des widgets
Liste_Sites = ['aucun','octaédriques','tétraédriques']
Liste_Vue =  ['Vue de face','Coupe diagonale','Coupe médiane']
Liste_Plan = ['Oui', 'Non']
type_site = ipw.Select(options=Liste_Sites,description='Type de sites :')
vue =  ipw.Select(options=Liste_Vue,description='Vue :')
plan =  ipw.Select(options=Liste_Plan,description='Plan Coupe :',value='Non')
rayon_site = ipw.FloatSlider(value=.1,min=.04,max=.2,step=0.02,description='Rayon invité :')


def maille(type_site,rayon_site,vue,plan) :

  coord_sites = sites(type_site)

  R_hote = 1 / (2*np.sqrt(2))

  if vue == 'Vue de face' :
    fig = plt.figure(figsize=(8.8,4), dpi=100)
  elif vue == 'Coupe médiane' :
    fig = plt.figure(figsize=(8.8,4), dpi=100)
  elif vue == 'Coupe diagonale' :
    fig = plt.figure(figsize=(12.8,4), dpi=100)

  #Sous-parties des graphiques
  ax1 = fig.add_subplot(121, projection='3d')
  ax2 = fig.add_subplot(122)

  #draw cube
  r = [0, 1]
  for s, e in combinations(np.array(list(product(r, r, r))), 2):
    if np.sum(np.abs(s-e)) == r[1]-r[0]:
      ax1.plot3D(*zip(s, e), color="green")

  #draw spheres
  def sphere(position, rayon) :
    us, vs = np.mgrid[0:2*np.pi:50j, 0:np.pi:50j]
    xs = rayon * np.cos(us)*np.sin(vs)
    ys = rayon * np.sin(us)*np.sin(vs)
    zs = rayon * np.cos(vs)
    return position[0] + xs, position[1] + ys, position[2] + zs

  #Ajout sphères sur vue 3D
  for k in coord_hote :
    R = R_hote
    ax1.plot_wireframe(sphere(k,R)[0], sphere(k,R)[1], sphere(k,R)[2], color="blue", alpha = .02)
    ax1.set_title("Vue 3D")

  for k in coord_sites :
    R = rayon_site
    ax1.plot_wireframe(sphere(k,R)[0], sphere(k,R)[1], sphere(k,R)[2], color="red", alpha = .1)


  #A-Situation sur une face

  if vue == 'Vue de face' :

    #ajout plan de coupe sur vue 3D
    if plan == 'Oui' :
      u, v = np.mgrid[0:1:30j, 0:1:30j]
      x = u
      y = 0*v
      z = v
      ax1.plot_wireframe(x,y,z, color="red", alpha = .2)

    #draw square
    ax2.add_artist(plt.Rectangle((0,0), 1, 1,fc='white',ec="green"))
    ax2.plot([0,1],[0,1],'k--',alpha=.3)
    ax2.plot([0,1],[1,0],'k--',alpha=.3)

    #draw circles
    for k in coord_hote :
      if k[2] == 0 :
        ax2.add_artist(plt.Circle((k[0],k[1]),R_hote,ec = 'blue',alpha = .5 ))

    for k in coord_sites :
      if k[2] == 0 :
        ax2.add_artist(plt.Circle((k[0],k[1]),rayon_site,ec = 'red',alpha = .8 ))

    ax2.set_xlim(0,1)
    ax2.set_ylim(0,1)
    ax2.set_title("Situation sur une face")
    ax2.get_xaxis().set_visible(False)
    ax2.get_yaxis().set_visible(False)



  #B-Situation sur coupe médiane

  elif vue == 'Coupe médiane' :

    #ajout plan sur vue 3D
    if plan == 'Oui' :
      u, v = np.mgrid[0:1:30j, 0:1:30j]
      x = 0*u + .5
      y = u
      z = v
      ax1.plot_wireframe(x,y,z, color="red", alpha = .5)

    #draw square
    ax2.add_artist(plt.Rectangle((0,0), 1, 1,fc='white',ec="green"))
    ax2.plot([0,1],[.5,.5],'k--',alpha=.3)
    ax2.plot([.5,.5],[0,1],'k--',alpha=.3)

    #draw circles
    for k in coord_hote :
      if k[1] == .5 :
        abscisse = (k[0]**2 + k[1]**2)**.5
        ax2.add_artist(plt.Circle((k[0],k[2]),R_hote,ec = 'blue',alpha = .3 ))

    for k in coord_sites :
      if k[1] == .5 :
        abscisse = (k[0]**2 + k[1]**2)**.5
        ax2.add_artist(plt.Circle((k[0],k[2]),rayon_site,ec = 'red',alpha = .8 ))

    ax2.set_xlim(0,1)
    ax2.set_ylim(0,1)
    ax2.set_title("Situation sur un plan médian")
    ax2.get_xaxis().set_visible(False)
    ax2.get_yaxis().set_visible(False)


  #C-Situation sur une diagonale

  elif vue == 'Coupe diagonale' :

    #ajout plan sur vue 3D
    if plan == 'Oui' :
      u, v = np.mgrid[0:1:30j, 0:1:30j]
      x = np.sqrt(v)
      y = np.sqrt(v)
      z = u
      ax1.plot_wireframe(x,y,z, color="red", alpha = .5)

    #draw rectangle
    ax2.add_artist(plt.Rectangle((0,0), 2**.5, 1,fc='white',ec="green"))

    # draw circles
    for k in coord_hote :
      if k[0] == k[1] :
        abscisse = (k[0]**2 + k[1]**2)**.5
        ax2.add_artist(plt.Circle((abscisse,k[2]),R_hote,ec = 'blue',alpha = .3 ))

    for k in coord_sites :
      if k[0] == k[1] :
        abscisse = (k[0]**2 + k[1]**2)**.5
        ax2.add_artist(plt.Circle((abscisse,k[2]),rayon_site,ec = 'red',alpha = .8 ))

    ax2.set_xlim(0,2**.5)
    ax2.set_ylim(0,1)
    ax2.set_title("Situation sur un plan diagonal")
    ax2.get_xaxis().set_visible(False)
    ax2.get_yaxis().set_visible(False)
    ax2.plot([0,2**.5],[0,1],'k--',alpha=.3)
    ax2.plot([0,2**.5],[1,0],'k--',alpha=.3)

  plt.show()


out = ipw.interactive_output(maille, {'type_site': type_site, 'rayon_site': rayon_site, 'vue': vue, 'plan': plan})

interactive_out_example = ipw.HBox([ipw.VBox([type_site, vue, plan, rayon_site]), out])
interactive_out_example

